<a href="https://colab.research.google.com/github/zamirsiddiqui/summarization/blob/master/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import numpy as np
import nltk
from nltk.corpus import stopwords #provides list of english stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Read files from github
url = 'https://media.githubusercontent.com/media/zamirsiddiqui/summarization/master/train.article.txt'
train_article = pd.read_table(url, error_bad_lines=False, header = None)

In [9]:
train_article.head()

,0
0,australia 's current account deficit shrunk by...
1,at least two people were killed in a suspected...
2,australian shares closed down #.# percent mond...
3,south korea 's nuclear envoy kim sook urged no...
4,south korea on monday announced sweeping tax r...


In [10]:
url = 'https://media.githubusercontent.com/media/zamirsiddiqui/summarization/master/train.title.txt'
train_title = pd.read_table(url, error_bad_lines=False, header = None)
train_title.head()

,0
0,australian current account deficit narrows sha...
1,at least two dead in southern philippines blast
2,australian stocks close down #.# percent
3,envoy urges north korea to restart nuclear dis...
4,skorea announces tax cuts to stimulate economy
